# 第25章：综合项目 - 文档分析系统

## 项目概述

本章将构建一个**自动化文档分析和总结系统**，展示批量处理和结构化输出的实际应用。

### 🎯 项目目标

构建一个智能文档分析系统，能够：
- 📄 批量处理多个文档
- 🎯 自动生成摘要
- 🔍 提取关键信息
- 📊 结构化输出结果
- 📝 生成分析报告

### 🛠 技术栈

| 组件 | 技术 | 作用 |
|------|------|------|
| **文档处理** | Document Loaders + Text Splitters | 加载和分割文档 |
| **结构化输出** | with_structured_output + Pydantic | 规范化数据格式 |
| **批处理** | batch() | 并行处理多个文档 |
| **LCEL Chains** | Runnable + Pipe (&#124;) | 构建处理流程 |
| **并发控制** | max_concurrency | 控制并发数量 |

### 📋 系统架构

```
多个文档输入
    ↓
文档加载器 (Document Loader)
    ↓
批量处理 (Batch Processing)
    ├─ 文档1 → 分析 → 结构化输出
    ├─ 文档2 → 分析 → 结构化输出
    ├─ 文档3 → 分析 → 结构化输出
    └─ 文档N → 分析 → 结构化输出
    ↓
聚合结果
    ↓
生成报告
```

### 💡 核心特性

1. **批量处理** - 高效处理多个文档
2. **结构化输出** - Pydantic模型确保数据一致性
3. **LCEL链** - 优雅的函数式编程风格
4. **并发控制** - 避免API限流
5. **错误处理** - 鲁棒的异常处理机制

### 🆚 vs 前面章节

| 对比项 | 第23章（RAG） | 第24章（Chatbot） | 第25章（文档分析） |
|--------|--------------|------------------|-------------------|
| **核心目标** | 文档问答 | 多轮对话 | 批量文档处理 |
| **处理模式** | 单次检索 | 交互式对话 | 批量并行处理 |
| **输出格式** | 自然语言 | 自然语言 | 结构化数据 |
| **关键技术** | Retriever | Checkpointer | batch() + Pydantic |
| **应用场景** | 知识问答 | 客服助手 | 数据分析、报告生成 |

---

## 学习收获

通过本项目，你将学会：
- ✅ 如何使用batch()并行处理数据
- ✅ 如何使用Pydantic定义结构化输出
- ✅ 如何使用LCEL构建处理链
- ✅ 如何控制并发避免限流
- ✅ 如何处理批量处理中的错误


In [1]:
# 环境配置
import os
import sys
import time

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_core.output_parsers import StrOutputParser
from typing import List, Optional
import json

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 步骤1：准备测试文档

创建一些示例文档用于分析。

In [2]:
print("【步骤1：准备测试文档】\n")

# 创建多个测试文档
documents = [
    {
        "id": "doc_001",
        "title": "人工智能技术发展报告",
        "content": """
人工智能(AI)技术在过去十年中取得了突破性进展。深度学习、自然语言处理和计算机视觉
等领域的创新推动了AI在各行各业的广泛应用。GPT系列模型的出现标志着大语言模型时代
的到来，使得机器能够理解和生成人类语言，在对话、写作、编程等任务上表现出色。

未来，AI技术将继续向更强的通用人工智能(AGI)方向发展，同时也需要关注AI伦理、
数据隐私和安全等重要议题。预计到2030年，AI将创造数万亿美元的经济价值。
        """
    },
    {
        "id": "doc_002",
        "title": "区块链技术应用白皮书",
        "content": """
区块链技术作为一种分布式账本技术，具有去中心化、不可篡改、透明可追溯等特点。
比特币和以太坊的成功展示了区块链在数字货币和智能合约领域的巨大潜力。

近年来，区块链应用已扩展到供应链管理、数字身份、金融服务等多个领域。NFT和
DeFi的兴起进一步推动了Web3生态的发展。然而，可扩展性、能源消耗和监管合规
仍是区块链大规模应用面临的主要挑战。
        """
    },
    {
        "id": "doc_003",
        "title": "量子计算前沿研究",
        "content": """
量子计算利用量子力学原理进行信息处理，在特定问题上具有指数级的计算优势。
Google、IBM等科技巨头正在积极研发量子计算机，并取得了量子优越性的里程碑成果。

量子计算有望在密码学、药物发现、材料科学、金融建模等领域带来革命性突破。
但量子纠错、量子比特稳定性等技术难题仍需克服。预计10-20年内，量子计算将
进入实用化阶段。
        """
    },
    {
        "id": "doc_004",
        "title": "5G与物联网融合发展",
        "content": """
5G网络的高速率、低延迟特性为物联网(IoT)应用提供了强大支持。智能城市、
工业互联网、自动驾驶等场景正在5G技术的推动下加速落地。

物联网设备数量预计将在未来五年内突破500亿台，形成万物互联的生态系统。
边缘计算和5G的结合将使实时数据处理成为可能，开创新的商业模式和服务形态。
安全性和隐私保护是物联网发展需要重点关注的问题。
        """
    },
    {
        "id": "doc_005",
        "title": "可持续能源技术趋势",
        "content": """
随着全球气候变化问题日益严峻，可持续能源技术成为各国重点发展方向。太阳能、
风能、氢能等清洁能源技术不断进步，成本持续下降，竞争力显著提升。

储能技术的突破解决了可再生能源的间歇性问题，电池技术和智能电网的发展推动了
能源转型。到2050年，预计可再生能源将占全球能源消费的70%以上，助力实现
碳中和目标。
        """
    }
]

print(f"✓ 准备了 {len(documents)} 个测试文档\n")
for doc in documents:
    print(f"  📄 {doc['id']}: {doc['title']}")
    print(f"     内容长度: {len(doc['content'])} 字符\n")

print("💡 这些文档涵盖了：")
print("  - 人工智能")
print("  - 区块链")
print("  - 量子计算")
print("  - 5G/物联网")
print("  - 可持续能源")

【步骤1：准备测试文档】

✓ 准备了 5 个测试文档

  📄 doc_001: 人工智能技术发展报告
     内容长度: 217 字符

  📄 doc_002: 区块链技术应用白皮书
     内容长度: 183 字符

  📄 doc_003: 量子计算前沿研究
     内容长度: 176 字符

  📄 doc_004: 5G与物联网融合发展
     内容长度: 178 字符

  📄 doc_005: 可持续能源技术趋势
     内容长度: 165 字符

💡 这些文档涵盖了：
  - 人工智能
  - 区块链
  - 量子计算
  - 5G/物联网
  - 可持续能源


## 步骤2：定义结构化输出模型

使用Pydantic定义文档分析的输出格式。

In [ ]:
print("【步骤2：定义结构化输出模型】\n")

# 定义文档分析结果的Pydantic模型
class DocumentAnalysis(BaseModel):
    """文档分析结果的结构化输出"""
    
    doc_id: str = Field(description="文档ID")
    title: str = Field(description="文档标题")
    summary: str = Field(description="文档摘要（3-5句话）")
    keywords: List[str] = Field(description="关键词列表（3-5个）")
    category: str = Field(description="文档类别（如：技术报告、白皮书、研究论文等）")
    main_topics: List[str] = Field(description="主要话题（2-4个）")
    sentiment: str = Field(description="情感倾向：积极/中立/消极")
    complexity_level: str = Field(description="复杂度：简单/中等/复杂")
    key_findings: Optional[List[str]] = Field(
        default=None,
        description="关键发现或结论（如有）"
    )
    
    class Config:
        json_schema_extra = {
            "example": {
                "doc_id": "doc_001",
                "title": "示例文档",
                "summary": "这是一个示例文档的摘要。",
                "keywords": ["关键词1", "关键词2"],
                "category": "技术报告",
                "main_topics": ["话题1", "话题2"],
                "sentiment": "积极",
                "complexity_level": "中等",
                "key_findings": ["发现1", "发现2"]
            }
        }

print("✓ 定义了结构化输出模型：DocumentAnalysis\n")
print("模型字段：")
for field_name, field_info in DocumentAnalysis.model_fields.items():
    print(f"  - {field_name}: {field_info.description}")

print("\n💡 使用Pydantic的优势：")
print("  ✓ 自动类型验证")
print("  ✓ 清晰的schema定义")
print("  ✓ 易于序列化/反序列化")
print("  ✓ 与LLM结构化输出完美集成")

【步骤2：定义结构化输出模型】

✓ 定义了结构化输出模型：DocumentAnalysis

模型字段：
  - doc_id: 文档ID
  - title: 文档标题
  - summary: 文档摘要（3-5句话）
  - keywords: 关键词列表（3-5个）
  - category: 文档类别（如：技术报告、白皮书、研究论文等）
  - main_topics: 主要话题（2-4个）
  - sentiment: 情感倾向：积极/中立/消极
  - complexity_level: 复杂度：简单/中等/复杂
  - key_findings: 关键发现或结论（如有）

💡 使用Pydantic的优势：
  ✓ 自动类型验证
  ✓ 清晰的schema定义
  ✓ 易于序列化/反序列化
  ✓ 与LLM结构化输出完美集成


/tmp/ipykernel_1000553/3794650406.py:4: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class DocumentAnalysis(BaseModel):


## 步骤3：构建LCEL分析链

使用LCEL（LangChain Expression Language）构建文档分析链。

In [5]:
print("【步骤3：构建LCEL分析链】\n")

# 1. 创建Prompt模板
print("=== 创建Prompt模板 ===\n")
analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个专业的文档分析专家。

你的任务是分析给定的文档并提取以下信息：
- 文档标题和ID
- 简洁的摘要（3-5句话）
- 关键词（3-5个）
- 文档类别
- 主要话题
- 情感倾向
- 复杂度级别
- 关键发现

请确保分析准确、客观、全面。"""),
    ("human", """请分析以下文档：

**文档ID**: {doc_id}
**标题**: {title}
**内容**:
{content}

请按照指定的格式返回分析结果。""")
])
print("✓ Prompt模板创建完成")
print()

# 2. 使用with_structured_output配置结构化输出
print("=== 配置结构化输出 ===\n")
structured_llm = model.with_structured_output(DocumentAnalysis)
print("✓ 已配置结构化输出（使用DocumentAnalysis模型）")
print()

# 3. 构建LCEL链
print("=== 构建LCEL链 ===\n")
analysis_chain = analysis_prompt | structured_llm
print("✓ LCEL链构建完成")
print("  结构: Prompt → LLM → Structured Output")
print()

print("🎉 文档分析链已就绪！")
print("\n💡 LCEL的优势：")
print("  - 使用管道符(|)连接组件")
print("  - 代码简洁易读")
print("  - 自动处理数据流转")
print("  - 支持流式和批量处理")

【步骤3：构建LCEL分析链】

=== 创建Prompt模板 ===

✓ Prompt模板创建完成

=== 配置结构化输出 ===

✓ 已配置结构化输出（使用DocumentAnalysis模型）

=== 构建LCEL链 ===

✓ LCEL链构建完成
  结构: Prompt → LLM → Structured Output

🎉 文档分析链已就绪！

💡 LCEL的优势：
  - 使用管道符(|)连接组件
  - 代码简洁易读
  - 自动处理数据流转
  - 支持流式和批量处理


## 步骤4：批量处理文档

使用`batch()`并行处理多个文档。

In [6]:
print("【步骤4：批量处理文档】\n")

import time

# 准备输入数据
inputs = [
    {"doc_id": doc["id"], "title": doc["title"], "content": doc["content"]}
    for doc in documents
]

print(f"准备批量处理 {len(inputs)} 个文档...\n")

# 方式1：使用batch()并行处理
print("=== 使用batch()并行处理 ===\n")
start_time = time.time()

results = analysis_chain.batch(
    inputs,
    config={"max_concurrency": 3}  # 控制并发数，避免限流
)

end_time = time.time()

print(f"✓ 批量处理完成！")
print(f"  处理文档数: {len(results)}")
print(f"  总耗时: {end_time - start_time:.2f} 秒")
print(f"  平均每篇: {(end_time - start_time) / len(results):.2f} 秒")
print()

# 显示部分结果
print("=== 分析结果预览 ===\n")
for i, result in enumerate(results[:2], 1):  # 显示前2个
    print(f"【文档 {i}】{result.title}")
    print(f"  类别: {result.category}")
    print(f"  摘要: {result.summary[:80]}...")
    print(f"  关键词: {', '.join(result.keywords)}")
    print(f"  情感: {result.sentiment} | 复杂度: {result.complexity_level}")
    print()

print("💡 batch()的优势：")
print("  ✓ 自动并行处理，提高效率")
print("  ✓ max_concurrency控制并发数")
print("  ✓ 相比串行处理节省大量时间")
print("  ✓ 适合处理大量独立任务")

【步骤4：批量处理文档】

准备批量处理 5 个文档...

=== 使用batch()并行处理 ===

✓ 批量处理完成！
  处理文档数: 5
  总耗时: 12.31 秒
  平均每篇: 2.46 秒

=== 分析结果预览 ===

【文档 1】人工智能技术发展报告
  类别: 技术报告
  摘要: 本文报告了人工智能技术在过去十年的突破性进展，特别是在深度学习和大语言模型领域的创新。GPT系列模型的出现推动了AI在语言理解和生成方面的能力提升。未来，AI将...
  关键词: 人工智能, 深度学习, 大语言模型, 通用人工智能, AI伦理
  情感: 积极 | 复杂度: 中等

【文档 2】区块链技术应用白皮书
  类别: 白皮书
  摘要: 本文介绍了区块链技术的核心特性及其在数字货币和智能合约领域的成功应用。区块链技术的应用范围已扩展到供应链管理、数字身份和金融服务等多个领域。NFT和DeFi的兴...
  关键词: 区块链, 数字货币, 智能合约, NFT, DeFi
  情感: 中立 | 复杂度: 中等

💡 batch()的优势：
  ✓ 自动并行处理，提高效率
  ✓ max_concurrency控制并发数
  ✓ 相比串行处理节省大量时间
  ✓ 适合处理大量独立任务


## 步骤5：生成分析报告

聚合所有分析结果，生成综合报告。

In [7]:
print("【步骤5：生成分析报告】\n")

# 统计分析
print("=== 数据统计 ===\n")
categories = {}
sentiments = {}
complexity_levels = {}
all_keywords = []

for result in results:
    # 统计类别
    categories[result.category] = categories.get(result.category, 0) + 1
    # 统计情感
    sentiments[result.sentiment] = sentiments.get(result.sentiment, 0) + 1
    # 统计复杂度
    complexity_levels[result.complexity_level] = complexity_levels.get(result.complexity_level, 0) + 1
    # 收集关键词
    all_keywords.extend(result.keywords)

print(f"文档类别分布:")
for cat, count in categories.items():
    print(f"  - {cat}: {count}篇")
print()

print(f"情感倾向分布:")
for sent, count in sentiments.items():
    print(f"  - {sent}: {count}篇")
print()

print(f"复杂度分布:")
for level, count in complexity_levels.items():
    print(f"  - {level}: {count}篇")
print()

# 生成JSON报告
print("=== 生成JSON报告 ===\n")
report = {
    "meta": {
        "total_documents": len(results),
        "analysis_date": time.strftime("%Y-%m-%d %H:%M:%S"),
        "processing_time": f"{end_time - start_time:.2f}秒"
    },
    "statistics": {
        "categories": categories,
        "sentiments": sentiments,
        "complexity_levels": complexity_levels,
        "unique_keywords": len(set(all_keywords))
    },
    "documents": [result.model_dump() for result in results]
}

# 保存报告
report_path = os.path.join(_project_root, "docs", "analysis_report.json")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"✓ 报告已保存: {report_path}")
print(f"  包含 {len(results)} 篇文档的完整分析")
print()

print("📊 报告内容：")
print("  - 元数据（总数、日期、耗时）")
print("  - 统计信息（类别、情感、复杂度）")
print("  - 每篇文档的详细分析结果")
print()

print("💡 应用场景：")
print("  - 批量文档分类和标注")
print("  - 内容审核和质量检查")
print("  - 知识库构建和管理")
print("  - 趋势分析和数据挖掘")

【步骤5：生成分析报告】

=== 数据统计 ===

文档类别分布:
  - 技术报告: 4篇
  - 白皮书: 1篇

情感倾向分布:
  - 积极: 4篇
  - 中立: 1篇

复杂度分布:
  - 中等: 5篇

=== 生成JSON报告 ===

✓ 报告已保存: /home/iip/tp/038-project/rag/docs/analysis_report.json
  包含 5 篇文档的完整分析

📊 报告内容：
  - 元数据（总数、日期、耗时）
  - 统计信息（类别、情感、复杂度）
  - 每篇文档的详细分析结果

💡 应用场景：
  - 批量文档分类和标注
  - 内容审核和质量检查
  - 知识库构建和管理
  - 趋势分析和数据挖掘


## 项目总结

### 🎉 完成的功能

本项目实现了一个完整的文档分析系统：

| 功能模块 | 实现方式 | 关键技术 |
|---------|---------|---------|
| **结构化输出** | Pydantic BaseModel | 类型安全的数据模型 |
| **LCEL链** | Prompt &#124; LLM &#124; Output | 函数式编程风格 |
| **批量处理** | batch() + max_concurrency | 并行处理提高效率 |
| **数据聚合** | Python统计 + JSON | 生成综合报告 |

### 🔑 关键技术点

#### 1. Pydantic结构化输出

```python
class DocumentAnalysis(BaseModel):
    doc_id: str = Field(description="文档ID")
    summary: str = Field(description="文档摘要")
    keywords: List[str] = Field(description="关键词")
    # ... 更多字段

# 使用with_structured_output
structured_llm = llm.with_structured_output(DocumentAnalysis)
```

**优势：**
- 自动类型验证
- 清晰的数据结构
- 易于序列化
- 与LLM无缝集成

#### 2. LCEL（LangChain Expression Language）

```python
# 使用管道符连接组件
chain = prompt | llm | output_parser

# 优雅简洁
result = chain.invoke(input_data)
```

**优势：**
- 代码简洁易读
- 组件可复用
- 支持流式处理
- 自动数据流转

#### 3. batch()批量处理

```python
results = chain.batch(
    inputs,
    config={"max_concurrency": 3}
)
```

**对比串行处理：**
- 串行: 5文档 × 2秒/篇 = 10秒
- 并行(3并发): 约 4秒（节省60%时间）

### 💡 性能优化策略

#### 1. 并发控制

```python
config={"max_concurrency": 3}
```

**原因：**
- 避免API限流
- 控制资源消耗
- 平衡速度和稳定性

**推荐值：**
- OpenAI: 3-5
- 自建服务: 10+

#### 2. 批量大小

**建议：**
- 小批量(10-50): 快速反馈
- 大批量(100+): 使用batch_as_completed()流式处理

#### 3. 错误处理

```python
# 使用try-except包裹batch
try:
    results = chain.batch(inputs, config={"max_concurrency": 3})
except Exception as e:
    # 处理失败的文档
    pass
```

### 🚀 扩展方向

#### 1. 更复杂的分析

```python
class AdvancedAnalysis(BaseModel):
    entities: List[str] = Field(description="命名实体")
    relationships: List[dict] = Field(description="实体关系")
    citations: List[str] = Field(description="引用来源")
    confidence_score: float = Field(description="分析置信度")
```

#### 2. 多步骤处理

```python
# 步骤1：提取
extraction_chain = extract_prompt | llm | ExtractSchema

# 步骤2：分析
analysis_chain = analysis_prompt | llm | AnalysisSchema

# 组合
full_chain = extraction_chain | analysis_chain
```

#### 3. 流式处理大文件

```python
# 对于超长文档
for chunk in split_document(long_doc):
    chunk_result = analysis_chain.invoke(chunk)
    # 合并结果
```

###📊 应用场景

1. **内容管理系统**
   - 自动标注和分类
   - 生成元数据
   - 构建搜索索引

2. **数据分析**
   - 批量处理调研报告
   - 提取关键指标
   - 趋势分析

3. **合规审查**
   - 批量审核文档
   - 检测敏感信息
   - 生成审核报告

4. **知识管理**
   - 自动摘要生成
   - 关键词提取
   - 知识图谱构建

### ⚠️ 注意事项

#### Q1: batch()失败怎么办？

**A:** 使用错误处理
```python
from langchain_core.runnables import RunnableConfig

# 方案1：忽略错误
config = RunnableConfig(max_concurrency=3, return_exceptions=True)

# 方案2：重试机制
# 自行实现重试逻辑
```

#### Q2: 如何处理不同长度的文档？

**A:** 分组处理
```python
short_docs = [d for d in docs if len(d["content"]) < 1000]
long_docs = [d for d in docs if len(d["content"]) >= 1000]

# 分别处理
short_results = chain.batch(short_docs, config={"max_concurrency": 5})
long_results = chain.batch(long_docs, config={"max_concurrency": 2})
```

#### Q3: 结构化输出验证失败？

**A:** 添加重试或降级
```python
# 方案1：使用ToolStrategy包装
from langchain.agents import ToolStrategy

response_format = ToolStrategy(
    DocumentAnalysis,
    retries=2  # 自动重试
)

# 方案2：降级到文本输出
try:
    result = structured_llm.invoke(input)
except ValidationError:
    result = llm.invoke(input)  # 降级到普通LLM
```

---

## 关键收获

✅ **Pydantic = 类型安全** - 确保输出格式一致性

✅ **LCEL = 优雅组合** - 函数式编程风格

✅ **batch() = 高效并行** - 大幅提升处理速度

✅ **max_concurrency = 资源控制** - 避免限流

✅ **结构化数据 = 易于处理** - 下游系统直接使用

---

## 三个项目对比

| 特性 | 第23章（RAG） | 第24章（Chatbot） | 第25章（文档分析） |
|------|--------------|------------------|-------------------|
| **目标** | 问答 | 对话 | 批量分析 |
| **输入** | 单个问题 | 多轮对话 | 批量文档 |
| **输出** | 自然语言 | 自然语言 | 结构化数据 |
| **状态** | 无状态 | 有状态 | 无状态 |
| **核心技术** | Retriever | Checkpointer | batch() + Pydantic |
| **并发** | 串行 | 串行 | 并行 |

🎉 恭喜！你已经掌握了LangChain应用开发的三大核心场景！
